In [72]:
import pandas as pd
import sqlalchemy

import pandas as pd
import sqlalchemy

# A new table we'll be working with
TABLE_NAME = "smi_data"

# Database credentials, which we created before
SQL_USERNAME = "kathyshen"
SQL_PASSWORD = "Helen71smc!"
SQL_DB_NAME = "digitaltools_db"

# Connection settings
SQL_DIALECT = "mysql"  # we are using MySQL
SQL_DRIVER = "pymysql"  # pymysql provides an interface between MySQL and Python

# The 
SQL_URL = "{}+{}://{}:{}@localhost:3306/{}".format(SQL_DIALECT,
                                                   SQL_DRIVER,
                                                   SQL_USERNAME,
                                                   SQL_PASSWORD,
                                                   SQL_DB_NAME)

# Connect to the DB
engine = sqlalchemy.create_engine(SQL_URL, echo=False)

In [73]:
data1 = pd.read_csv("/Users/katherine/Public/study/20Fall/digital-tools-for-finance/SMI_Historical_Data.csv", parse_dates=[0], index_col=None)
data1.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,2020-12-11,"10,296.89","10,372.50","10,387.23","10,293.50",-,-0.95%
1,2020-12-10,"10,395.96","10,462.51","10,485.45","10,375.95",-,-0.33%
2,2020-12-09,"10,430.02","10,412.84","10,469.15","10,378.52",43.14M,0.35%
3,2020-12-08,"10,394.10","10,387.44","10,407.62","10,265.31",48.95M,0.18%
4,2020-12-07,"10,375.38","10,375.02","10,422.96","10,337.42",49.31M,0.10%


In [74]:
data1.rename(columns={"Date": "_date"}, inplace=True)

In [75]:
data1.drop("Change %", axis=1, inplace=True, errors="ignore")
data1.dtypes

_date    datetime64[ns]
Price            object
Open             object
High             object
Low              object
Vol.             object
dtype: object

In [76]:
data1.head()

,_date,Price,Open,High,Low,Vol.
0,2020-12-11,"10,296.89","10,372.50","10,387.23","10,293.50",-
1,2020-12-10,"10,395.96","10,462.51","10,485.45","10,375.95",-
2,2020-12-09,"10,430.02","10,412.84","10,469.15","10,378.52",43.14M
3,2020-12-08,"10,394.10","10,387.44","10,407.62","10,265.31",48.95M
4,2020-12-07,"10,375.38","10,375.02","10,422.96","10,337.42",49.31M


In [77]:
#drop Vol. column
data1.drop("Vol.", axis=1, inplace=True, errors="ignore")


statements = [
    "DROP TABLE IF EXISTS {};".format(TABLE_NAME),
    
    "CREATE TABLE {} ("
        "_date date,"
        "Price VARCHAR(10)," 
        "Open VARCHAR(10),"
        "High VARCHAR(10),"
        "Low VARCHAR(10),"
        "PRIMARY KEY (_date,Price));".format(TABLE_NAME) 
    ]

for s in statements:
    engine.execute(s)


data1.to_sql(con=engine, name=TABLE_NAME, **{"if_exists": "append", "index": False})

In [78]:
df = pd.read_sql(sql="SELECT * FROM {};".format(TABLE_NAME), con=engine)
df.head()

,_date,Price,Open,High,Low
0,2020-02-03,"10,664.95","10,650.02","10,694.76","10,598.12"
1,2020-02-04,"10,800.64","10,707.44","10,800.64","10,693.32"
2,2020-02-05,"10,994.15","10,789.72","11,006.72","10,781.57"
3,2020-02-06,"11,012.36","11,040.17","11,061.18","10,979.82"
4,2020-02-07,"11,001.53","10,995.65","11,015.61","10,958.45"


In [98]:
class DataBaseAPI:
    """A wrapper for saving to and retrieving data from an SQL database using
    SQLAlchemy. Nothing fancy here: saving is carried by the
    'pd.DataFrame.to_sql()' method, that is, entire dataframes are dumped into
    SQL tables.

    """

    def __init__(self, engine):
        """Create an instance of the class.

        Parameters
        ----------
        engine: sqlalchemy.engine.base.Engine
            providing connection to the database

        """
        self.engine = engine

    def save_data_to_table(self, data, table_name, **to_sql_kwargs):
        """Saves pandas dataframe to SQL table.

        Parameters
        ----------
        data: pd.DataFrame
            of data to be saved to the database
        table_name: str
            name of the table in the database whe
        to_sql_kwargs: dict
            with keyword arguments of the 'pd.DataFrame.to_sql()' method

        Returns
        -------
        Nothing; just saves the data

        """
        data.to_sql(con=self.engine, name=table_name, **to_sql_kwargs)
        
    def get_data(self, query, **read_sql_kwargs):
        """Wrapper around pd.read_sql running an SQL query and returning a
        dataframe with requested data.

        Parameters
        ----------
        query: str
            SQL query

        Returns
        -------
        data: pd.DataFrame
            corresponding to the 'query'
        read_sql_kwargs: dict
            of keyword arguments of the 'pd.read_sql()' function

        """
        return pd.read_sql(sql=query, con=self.engine, **read_sql_kwargs)


In [100]:
db_api = DataBaseAPI(engine=engine)

# Create a new table to accomodate the data
statements = [
    "DROP TABLE IF EXISTS {};".format(TABLE_NAME),
    
    "CREATE TABLE {} ("
        "_date date,"
        "Price VARCHAR(10)," 
        "Open VARCHAR(10),"
        "High VARCHAR(10),"
        "Low VARCHAR(10),"
        "PRIMARY KEY (_date,Price));".format(TABLE_NAME)
    ]

for s in statements:
    db_api.engine.execute(s)

# Store the frame
db_api.save_data_to_table(data=data1, table_name=TABLE_NAME, **{"if_exists": "append", "index": False})


In [101]:
print(data1.head())

       _date      Price       Open       High        Low
0 2020-12-11  10,296.89  10,372.50  10,387.23  10,293.50
1 2020-12-10  10,395.96  10,462.51  10,485.45  10,375.95
2 2020-12-09  10,430.02  10,412.84  10,469.15  10,378.52
3 2020-12-08  10,394.10  10,387.44  10,407.62  10,265.31
4 2020-12-07  10,375.38  10,375.02  10,422.96  10,337.42
